# 05 - Telecom Capacity Forecasting

Predict future network traffic load per cell using LightGBM time-series regression.  
The model ingests hourly KPI snapshots (traffic, PRB utilization, throughput, latency, etc.) and forecasts `traffic_load_gb` to support proactive capacity planning.

## 1. Setup & Configuration

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("notebook")
sns.set_style("whitegrid")
sns.set_palette("husl")

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

In [ ]:
import sys
from pathlib import Path

# Add project source to path so we can import the project modules
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

print(f"Project root : {PROJECT_ROOT}")
print(f"Raw data dir : {DATA_RAW}")

## 2. Data Loading & Validation

In [ ]:
df = pd.read_parquet(DATA_RAW / "synthetic_data.parquet")

print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

In [ ]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

print(f"Time range : {df['timestamp'].min()} --> {df['timestamp'].max()}")
print(f"Unique cells: {df['cell_id'].nunique()}")
print(f"Cell types  : {df['cell_type'].value_counts().to_dict()}")
print(f"Area types  : {df['area_type'].value_counts().to_dict()}")
print(f"\nMissing values:\n{df.isnull().sum()[df.isnull().sum() > 0]}")
if df.isnull().sum().sum() == 0:
    print("No missing values found.")

In [ ]:
df.describe()

## 3. Exploratory Data Analysis

In [ ]:
# Time-series plot for a sample of cells
sample_cells = df["cell_id"].unique()[:4]

fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=True)
for ax, cell in zip(axes.ravel(), sample_cells):
    cell_df = df[df["cell_id"] == cell].sort_values("timestamp")
    ax.plot(cell_df["timestamp"], cell_df["traffic_load_gb"], linewidth=0.8)
    ax.set_title(f"{cell}  ({cell_df['area_type'].iloc[0]})", fontsize=11)
    ax.set_ylabel("Traffic Load (GB)")
fig.suptitle("Hourly Traffic Load for Sample Cells", fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Seasonal decomposition for one cell
from statsmodels.tsa.seasonal import seasonal_decompose

cell_ts = (
    df[df["cell_id"] == sample_cells[0]]
    .sort_values("timestamp")
    .set_index("timestamp")["traffic_load_gb"]
)
decomposition = seasonal_decompose(cell_ts, model="additive", period=24)

fig, axes = plt.subplots(4, 1, figsize=(14, 10), sharex=True)
decomposition.observed.plot(ax=axes[0], title="Observed")
decomposition.trend.plot(ax=axes[1], title="Trend")
decomposition.seasonal.plot(ax=axes[2], title="Seasonal (24h)")
decomposition.resid.plot(ax=axes[3], title="Residual")
fig.suptitle(f"Seasonal Decomposition: {sample_cells[0]}", fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Weekday vs weekend patterns
df["hour"] = df["timestamp"].dt.hour
df["is_weekend"] = df["timestamp"].dt.dayofweek >= 5

hourly_agg = (
    df.groupby(["hour", "is_weekend"])["traffic_load_gb"]
    .mean()
    .reset_index()
)
hourly_agg["day_type"] = hourly_agg["is_weekend"].map({True: "Weekend", False: "Weekday"})

fig, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(data=hourly_agg, x="hour", y="traffic_load_gb", hue="day_type", marker="o", ax=ax)
ax.set_title("Average Traffic Load by Hour: Weekday vs Weekend")
ax.set_xlabel("Hour of Day")
ax.set_ylabel("Mean Traffic Load (GB)")
ax.set_xticks(range(24))
plt.tight_layout()
plt.show()

In [ ]:
# Traffic distribution by area type
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(data=df, x="traffic_load_gb", hue="area_type", kde=True,
             stat="density", common_norm=False, alpha=0.5, ax=axes[0])
axes[0].set_title("Traffic Load Distribution by Area Type")

sns.boxplot(data=df, x="area_type", y="traffic_load_gb", ax=axes[1])
axes[1].set_title("Traffic Load Box Plot by Area Type")

plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix of numeric columns
numeric_cols = ["traffic_load_gb", "connected_users", "prb_utilization",
                "avg_throughput_mbps", "avg_latency_ms", "avg_sinr_db"]

fig, ax = plt.subplots(figsize=(9, 7))
corr = df[numeric_cols].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap="coolwarm",
            center=0, square=True, ax=ax)
ax.set_title("Feature Correlation Matrix")
plt.tight_layout()
plt.show()

## 4. Feature Engineering

In [ ]:
from capacity_forecasting.features import FeatureEngineer

# Reload raw data to avoid leaking EDA columns
df_raw = pd.read_parquet(DATA_RAW / "synthetic_data.parquet")
df_raw["timestamp"] = pd.to_datetime(df_raw["timestamp"])
df_raw = df_raw.sort_values(["cell_id", "timestamp"]).reset_index(drop=True)

fe = FeatureEngineer()
print(f"Raw shape: {df_raw.shape}")

In [ ]:
# Step-by-step: lag features
df_feat = fe.create_lag_features(
    df_raw, group_col="cell_id", target_col="traffic_load_gb", lags=[1, 24, 168]
)
print("Lag feature columns added:")
print([c for c in df_feat.columns if "lag" in c])
df_feat[["cell_id", "timestamp", "traffic_load_gb",
         "traffic_load_gb_lag_1h", "traffic_load_gb_lag_24h",
         "traffic_load_gb_lag_168h"]].head(10)

In [ ]:
# Step-by-step: rolling aggregates
df_feat = fe.create_rolling_aggregates(
    df_feat,
    group_col="cell_id",
    value_cols=["traffic_load_gb", "prb_utilization"],
    windows=[24, 168],
)
rolling_cols = [c for c in df_feat.columns if "rolling" in c]
print(f"Rolling aggregate columns ({len(rolling_cols)}): {rolling_cols}")

In [ ]:
# Run the full pipeline (temporal, lags, interactions, encoding, missing-value handling)
df_engineered = fe.pipeline(df_raw)
print(f"\nEngineered shape: {df_engineered.shape}")
print(f"Columns: {list(df_engineered.columns)}")

## 5. Model Training

In [ ]:
from capacity_forecasting.models import LightGBMForecaster, print_metrics

# Prepare modelling DataFrame: drop non-numeric / identifier columns
drop_cols = [c for c in ["cell_id", "timestamp"] if c in df_engineered.columns]
df_model = df_engineered.drop(columns=drop_cols)

# Ensure all columns are numeric (pipeline should have encoded categoricals)
print(f"Modelling shape: {df_model.shape}")
print(f"Target: traffic_load_gb")
df_model.dtypes

In [ ]:
# Chronological train/test split
forecaster = LightGBMForecaster()
X_train, X_test, y_train, y_test = forecaster.prepare_time_series_data(
    df_model, target_col="traffic_load_gb", test_ratio=0.2
)

In [ ]:
# Train the LightGBM forecaster
forecaster.train(X_train, y_train)

## 6. Evaluation & Metrics

In [ ]:
# Compute regression metrics (MAPE, RMSE, MAE, R2)
metrics = forecaster.evaluate(X_test, y_test)
print_metrics(metrics, title="Capacity Forecaster Performance")

In [ ]:
# Forecast vs Actual for sample cells
y_pred = forecaster.predict(X_test)

# Reconstruct cell mapping from the test portion of the engineered frame
test_idx = X_test.index
test_cells = df_engineered.loc[test_idx, "cell_id"].values if "cell_id" in df_engineered.columns else None

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
if test_cells is not None:
    unique_test_cells = pd.Series(test_cells).unique()[:4]
    for ax, cell in zip(axes.ravel(), unique_test_cells):
        mask = test_cells == cell
        ax.plot(y_test.values[mask][:200], label="Actual", alpha=0.8)
        ax.plot(y_pred[mask][:200], label="Predicted", alpha=0.8, linestyle="--")
        ax.set_title(cell)
        ax.set_ylabel("Traffic Load (GB)")
        ax.legend(fontsize=9)
else:
    axes[0, 0].plot(y_test.values[:400], label="Actual", alpha=0.8)
    axes[0, 0].plot(y_pred[:400], label="Predicted", alpha=0.8, linestyle="--")
    axes[0, 0].legend()

fig.suptitle("Forecast vs Actual Traffic Load (Test Set)", fontsize=13, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Error distribution
errors = y_test.values - y_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(errors, kde=True, bins=60, ax=axes[0])
axes[0].axvline(0, color="red", linestyle="--", linewidth=1)
axes[0].set_title("Prediction Error Distribution")
axes[0].set_xlabel("Error (Actual - Predicted)")

axes[1].scatter(y_test, y_pred, alpha=0.15, s=8)
lims = [min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())]
axes[1].plot(lims, lims, "r--", linewidth=1, label="Perfect prediction")
axes[1].set_title("Actual vs Predicted")
axes[1].set_xlabel("Actual Traffic Load (GB)")
axes[1].set_ylabel("Predicted Traffic Load (GB)")
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. Interpretation

In [ ]:
# SHAP-based feature importance
import shap

explainer = shap.TreeExplainer(forecaster.model)

# Use a subsample for speed
X_explain = X_test.sample(n=min(1000, len(X_test)), random_state=42)
shap_values = explainer.shap_values(X_explain)

shap.summary_plot(shap_values, X_explain, show=False, max_display=15)
plt.title("SHAP Feature Importance (Top 15)")
plt.tight_layout()
plt.show()

In [ ]:
# Built-in LightGBM feature importance
importance_df = forecaster.get_feature_importance()

fig, ax = plt.subplots(figsize=(10, 8))
top_n = importance_df.head(15)
sns.barplot(data=top_n, y="feature", x="importance", ax=ax)
ax.set_title("LightGBM Feature Importance (Top 15)")
ax.set_xlabel("Importance (split)")
plt.tight_layout()
plt.show()

In [ ]:
# Lag feature analysis: how much do lag features contribute?
lag_features = [c for c in importance_df["feature"] if "lag" in c]
lag_importance = importance_df[importance_df["feature"].isin(lag_features)]
total_importance = importance_df["importance"].sum()
lag_share = lag_importance["importance"].sum() / total_importance * 100

print(f"Lag features account for {lag_share:.1f}% of total feature importance.")
print("\nLag feature breakdown:")
print(lag_importance.to_string(index=False))

## 8. Business Insights & Conclusions

In [ ]:
# Peak hour prediction accuracy
is_peak = ((X_test["hour"] >= 9) & (X_test["hour"] <= 11)) | \
          ((X_test["hour"] >= 18) & (X_test["hour"] <= 21)) \
          if "hour" in X_test.columns else pd.Series([False] * len(X_test))

if is_peak.any():
    peak_errors = np.abs(y_test[is_peak].values - y_pred[is_peak])
    offpeak_errors = np.abs(y_test[~is_peak].values - y_pred[~is_peak])
    print(f"Peak hour MAE   : {peak_errors.mean():.4f} GB")
    print(f"Off-peak MAE    : {offpeak_errors.mean():.4f} GB")
    print(f"Peak MAPE       : {(peak_errors / np.maximum(y_test[is_peak].values, 1e-6)).mean() * 100:.2f}%")
    print(f"Off-peak MAPE   : {(offpeak_errors / np.maximum(y_test[~is_peak].values, 1e-6)).mean() * 100:.2f}%")
else:
    print("Peak-hour column not available in test features.")

In [ ]:
# Capacity planning: identify cells likely to exceed threshold
CAPACITY_THRESHOLD_GB = df["traffic_load_gb"].quantile(0.95)

predicted_over_capacity = y_pred > CAPACITY_THRESHOLD_GB
actual_over_capacity = y_test.values > CAPACITY_THRESHOLD_GB

tp = (predicted_over_capacity & actual_over_capacity).sum()
fp = (predicted_over_capacity & ~actual_over_capacity).sum()
fn = (~predicted_over_capacity & actual_over_capacity).sum()

precision_cap = tp / max(tp + fp, 1)
recall_cap = tp / max(tp + fn, 1)

print(f"Capacity threshold (P95): {CAPACITY_THRESHOLD_GB:.2f} GB")
print(f"High-traffic detection precision: {precision_cap:.2%}")
print(f"High-traffic detection recall   : {recall_cap:.2%}")
print(f"\nThis means the model correctly flags {recall_cap:.0%} of true")
print(f"high-traffic periods, enabling proactive capacity upgrades.")

In [ ]:
# Summary table
summary = pd.DataFrame({
    "Metric": ["MAPE (%)", "RMSE (GB)", "MAE (GB)", "R-squared",
               "High-traffic Precision", "High-traffic Recall"],
    "Value": [
        f"{metrics['mape_pct']:.2f}",
        f"{metrics['rmse']:.4f}",
        f"{metrics['mae']:.4f}",
        f"{metrics['r2']:.4f}",
        f"{precision_cap:.2%}",
        f"{recall_cap:.2%}",
    ]
})
print(summary.to_string(index=False))

### Key Business Insights

1. **Lag features dominate**: The 1-hour and 24-hour lag features are the strongest predictors, confirming that recent and same-hour-yesterday traffic are the best signals for short-term forecasting.

2. **Peak-hour accuracy**: The model maintains strong accuracy during peak hours (morning 9-11, evening 18-21), which is precisely when capacity planning matters most.

3. **Area-type differentiation**: Urban cells carry significantly higher traffic loads than suburban/rural cells, and the model captures these structural differences through interaction and categorical features.

4. **Proactive capacity planning**: With high recall on above-threshold traffic predictions, network planners can schedule capacity upgrades 24-168 hours ahead of anticipated congestion.

5. **Special event detection**: The model's error distribution shows heavier tails for extreme events (special event multiplier in data), suggesting a secondary anomaly detection layer could further improve operational response.

### Recommended Next Steps

- Deploy as a rolling 24-hour forecast service per cell, refreshed hourly.
- Integrate with the anomaly detection system (Project 03) to flag unexpected surges.
- Extend the forecast horizon to 7 days for infrastructure procurement decisions.
- Add weather and event calendar features for improved special-event handling.